In [1]:
import sys
sys.path.insert(0, '..')

In [3]:
from src.data_processors_lib import DeleteUsersByPathLength, DeleteUsersByPathLengthParams

from src.backend.callback import list_dataprocessors
list_dataprocessors.list_dataprocessor({})

/Users/alexeyavramenko/projects/rete/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:73: UserWarning: Callable func was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()
/Users/alexeyavramenko/projects/rete/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:73: UserWarning: Callable negative_function was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()
/Users/alexeyavramenko/projects/rete/retentioneering-tools-new-arch/examples/../src/params_model/params_model.py:73: UserWarning: Callable positive_function was excluded from schema since JSON schema has no equivalent type.
  params_schema: dict[str, Any] = cls.schema()


[{'name': 'CollapseLoops',
  'params': [{'name': 'suffix',
    'widget': 'enum',
    'default': 'loop',
    'optional': True,
    'params': ['loop', 'count']},
   {'name': 'timestamp_aggregation_type',
    'widget': 'enum',
    'default': 'max',
    'optional': True,
    'params': ['max', 'min', 'mean']}]},
 {'name': 'DeleteUsersByPathLength',
  'params': [{'optional': True, 'name': 'events_num', 'widget': 'integer'},
   {'default': None,
    'widget': 'time_widget',
    'params': [{'widget': 'float'},
     {'params': ['Y',
       'M',
       'W',
       'D',
       'h',
       'm',
       's',
       'ms',
       'us',
       'μs',
       'ns',
       'ps',
       'fs',
       'as'],
      'widget': 'enum'}],
    'optional': True,
    'name': 'cutoff'}]},
 {'name': 'FilterEvents',
  'params': [{'default': None,
    'widget': 'function',
    'optional': True,
    'name': 'func'}]},
 {'name': 'GroupEvents',
  'params': [{'optional': False, 'name': 'event_name', 'widget': 'string'},
   {

In [4]:
import pandas as pd

# download_url = 'https://drive.google.com/uc?id=1tY-4xg6m_dv6IaVPIcd4oC1bK1lW5Tr9&export=download&confirm=t'
# df0 = pd.read_csv(download_url, compression='gzip')
df0 = pd.DataFrame(data=[], columns=['event', 'timestamp', 'user_id'])
df0

,event,timestamp,user_id


In [5]:
from src.eventstream.schema import RawDataSchema, EventstreamSchema
from src.eventstream.eventstream import Eventstream
from src.graph.p_graph import PGraph, EventsNode
from src.data_processors_lib import CollapseLoops, CollapseLoopsParams
from src.data_processors_lib import DeleteUsersByPathLength, DeleteUsersByPathLengthParams
from src.data_processors_lib import FilterEvents, FilterEventsParams
from src.data_processors_lib import GroupEvents, GroupEventsParams
from src.data_processors_lib import NewUsersEvents, NewUsersParams
from src.data_processors_lib import SplitSessions, SplitSessionsParams
from src.data_processors_lib import StartEndEvents, StartEndEventsParams
from src.data_processors_lib import TruncatePath, TruncatePathParams
from src.data_processors_lib import TruncatedEvents, TruncatedEventsParams
from src.graph.p_graph import PGraph, EventsNode
import inspect


raw_data_schema = RawDataSchema(
    event_name='event', 
    event_timestamp='timestamp', 
    user_id='user_id'
)

stream = Eventstream(
    raw_data=df0,
    raw_data_schema=raw_data_schema,
    schema=EventstreamSchema()
)

TARGET_EVENT = 'finances/deposit/<payment_name>/success'

def users_with_target_event(df, schema) -> pd.DataFrame:
    target_users = df[df['event_name'] == TARGET_EVENT]['user_id'].unique()
    return df['user_id'].isin(target_users)

def first_session_filter(df, schema)  -> pd.DataFrame:
    return df['session_id'].str.endswith('_1')

def new_and_not_truncated_users(df, schema)  -> pd.DataFrame:
    truncated_users = df[(df['event_name'] == 'truncated_right')]['user_id'].unique()
    new_users = df[(df['event_name'] == 'new_user')]['user_id'].unique()
    target_users = np.setdiff1d(new_users, truncated_users)
    return df['user_id'].isin(target_users)

def first_session_filter(df, schema)  -> pd.DataFrame:
    return df['session_id'].str.endswith('_1')

def create_graph():
    graph = PGraph(source_stream=stream)

    node0 = EventsNode(CollapseLoops(params=CollapseLoopsParams(**{})))
    node1 = EventsNode(StartEndEvents(params=StartEndEventsParams(**{})))
    node2 = EventsNode(NewUsersEvents(params=NewUsersParams(new_users_list="all")))
    node3 = EventsNode(TruncatedEvents(params=TruncatedEventsParams(right_truncated_cutoff=(12, 'D'))))
    node4 = EventsNode(FilterEvents(params=FilterEventsParams(func=new_and_not_truncated_users)))
    node5 = EventsNode(SplitSessions(params=SplitSessionsParams(
        session_cutoff=(1, 'h'),
        session_col='session_id'
    )))
    node6 = EventsNode(FilterEvents(params=FilterEventsParams(func=first_session_filter)))

    node7 = EventsNode(FilterEvents(params=FilterEventsParams(func=users_with_target_event)))
    node8 = EventsNode(TruncatePath(params=TruncatePathParams(drop_after=TARGET_EVENT)))
    node9 = EventsNode(FilterEvents(params=FilterEventsParams(func=first_session_filter)))

    graph.add_node(node=node0, parents=[graph.root])
    graph.add_node(node=node1, parents=[graph.root])
    graph.add_node(node=node2, parents=[node1])
    graph.add_node(node=node3, parents=[node2])
    graph.add_node(node=node4, parents=[node3])
    graph.add_node(node=node5, parents=[node4])

    graph.add_node(node=node6, parents=[node5])
    graph.add_node(node=node7, parents=[node6])

    graph.add_node(node=node8, parents=[node5])
    graph.add_node(node=node9, parents=[node8])

    return graph




In [6]:


graph = create_graph()

graph.display()


In [9]:
graph.display()

In [61]:
from __future__ import annotations

import inspect
import types
from dataclasses import dataclass, field
from typing import Any, Callable, Type, Union

from src.constants import DATETIME_UNITS_LIST
from src.exceptions.widget import ParseReteFuncError

def _serialize(value: Callable) -> str:
    try:
        code = inspect.getsource(value)
        return code
    except OSError:
        return ""

def _parse(value: str) -> Callable:  # type: ignore
    try:
        code_obj = compile(value, "<string>", "exec")
    except:
        raise ParseReteFuncError("parsing error. You must implement a python function here")

    new_func_type = None

    for i in code_obj.co_consts:
        try:
            new_func_type = types.FunctionType(i, {})
        except Exception as err:
            continue

    if new_func_type is None:
        raise ParseReteFuncError("parsing error. You must implement a python function here")

    return new_func_type

In [62]:
def first_session_filter(df, schema)  -> pd.DataFrame:
    return df['session_id'].str.endswith('_1')

_parse(_serialize(first_session_filter))


<function first_session_filter(df, schema)>

In [8]:
class Test():
    @classmethod
    def __init_subclass__(cls, **kwargs) -> None:
        super().__init_subclass__(**kwargs)
        print("init subclass called!")

class Inh(Test):
  def __init__(self):
    self.t = 3

  

init subclass called!
